In [1]:
!pip install pymupdf groq python-docx -q
print("✅ Libraries installed!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 18.7 MB/s eta 0:00:00
✅ Libraries installed!


In [2]:
import fitz
from groq import Groq
import json

GROQ_API_KEY = "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"

client = Groq(api_key=GROQ_API_KEY)
print("✅ API connected!")

✅ API connected!


In [3]:
def extract_text(filename):
    doc = fitz.open(filename)
    text = ""
    for page in doc:
        text += page.get_text()
    doc.close()
    return text

text_inspection = extract_text("Sample Report.pdf")
text_thermal = extract_text("Thermal Images.pdf")

print(f"✅ Inspection Report: {len(text_inspection)} characters")
print(f"✅ Thermal Report: {len(text_thermal)} characters")
print("\n--- Sample Inspection Text ---")
print(text_inspection[:500])

✅ Inspection Report: 7375 characters
✅ Thermal Report: 6147 characters

--- Sample Inspection Text ---
 
 
Inspection Form 
 
Complete 
 
Score 
85.71% 
Flagged items 
1 Actions 
0 
 
Customer Name 
Mobile: 
Email: 
 
 
Address: 
 
 
Property Age (In years): 
Floors: 
11 
 
Previous Structural audit done 
No 
Previous Repair work done 
No 
Inspection Date and Time: 
27.09.2022 14:28 IST 
Inspected By: 
Krushna & Mahesh 
Property Type: 
Flat 
 
 
 
 
Checklists / Inspection Checklists 
 
Checklist: 
 
 
 
Flagged items 
1 flagged 
WC 
External wall 
 
 
 
 
 
 
 
 
Impacted Areas/Rooms 
Hall, Bedr


In [4]:
def llm_extract(prompt, text, max_chars=4000):
    text_chunk = text[:max_chars]

    chat = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": "You are a precise data extractor. Output ONLY valid JSON. No explanations."},
            {"role": "user", "content": f"{prompt}\n\nText to extract from:\n{text_chunk}"}
        ],
        temperature=0.1
    )

    try:
        return json.loads(chat.choices[0].message.content)
    except:
        return {"error": "JSON parsing failed", "raw": chat.choices[0].message.content}

print("Extracting from Inspection Report...")
inspection_data = llm_extract(
    """Extract these fields as JSON:
    {
      "areas": ["list of mentioned areas/rooms"],
      "observations": ["list of issues/problems found"],
      "details": ["any specific measurements or conditions"]
    }""",
    text_inspection
)

print("Extracting from Thermal Report...")
thermal_data = llm_extract(
    """Extract these fields as JSON:
    {
      "areas": ["list of areas with thermal readings"],
      "temperatures": {"area": "temperature value"},
      "findings": ["list of thermal observations"]
    }""",
    text_thermal
)

print("✅ Extraction complete!")
print("\n--- Inspection Data ---")
print(json.dumps(inspection_data, indent=2))
print("\n--- Thermal Data ---")
print(json.dumps(thermal_data, indent=2))

Extracting from Inspection Report...
Extracting from Thermal Report...
✅ Extraction complete!

--- Inspection Data ---
{
  "areas": [
    "Hall",
    "Bedroom",
    "Kitchen",
    "Master Bedroom",
    "Parking Area",
    "Common Bathroom"
  ],
  "observations": [
    "Hall Skirting level Dampness",
    "Bedroom Skirting level Dampness",
    "Kitchen Skirting level Dampness",
    "Master Bedroom Skirting level Dampness",
    "Master bedroom wall damness",
    "Parking Area seepage",
    "Common Bathroom Ceiling Dampness",
    "Condition of leakage at adjacent walls",
    "Condition of leakage below WC",
    "Leakage during: All time",
    "Leakage due to concealed plumbing",
    "Leakage due to damage in Nahani trap/Brickbat coba under tile flooring",
    "Gaps/Blackish dirt Observed in tile joints",
    "Gaps around Nahani Trap Joints",
    "Tiles Broken/Loosed anywhere",
    "Loose Plumbing joints/rust around joints and edges (Flush Tank/shower/angle cock/bibcock, washbasin, etc)",
 

In [5]:
merge_prompt = f"""
You are an expert property inspector. Merge these two inspection datasets into a final DDR report.

**INSPECTION DATA:**
{json.dumps(inspection_data, indent=2)}

**THERMAL DATA:**
{json.dumps(thermal_data, indent=2)}

**STRICT RULES:**
1. Do NOT invent any facts not in the data
2. If information conflicts, write "Conflict: [describe]"
3. If information is missing, write "Not Available"
4. Use simple, client-friendly language
5. Combine overlapping observations logically

**Output EXACTLY this JSON structure:**
{{
  "property_issue_summary": "Brief 2-3 sentence summary of main problems",
  "area_wise_observations": "Detailed observations organized by area/room",
  "probable_root_cause": "Most likely causes for each issue",
  "severity_assessment": "Low/Medium/High - with reasoning for each issue",
  "recommended_actions": "Prioritized list of actions client should take",
  "additional_notes": "Any important context or warnings",
  "missing_info": "List gaps in data or write 'None identified'"
}}
"""

print("Generating DDR report...")
ddr_json = llm_extract("Generate the DDR report:", merge_prompt, max_chars=8000)

print("✅ DDR Generated!")
print(json.dumps(ddr_json, indent=2))

Generating DDR report...
✅ DDR Generated!
{
  "property_issue_summary": "The property has several issues including dampness, leakage, and thermal anomalies. The main problems are related to the skirting, walls, and floors. Immediate action is required to prevent further damage.",
  "area_wise_observations": {
    "Hall": [
      "Skirting level Dampness",
      "Conflict: No thermal data available for this area"
    ],
    "Bedroom": [
      "Skirting level Dampness",
      "Conflict: No thermal data available for this area"
    ],
    "Kitchen": [
      "Skirting level Dampness",
      "Conflict: No thermal data available for this area"
    ],
    "Master Bedroom": [
      "Skirting level Dampness",
      "Master bedroom wall damness",
      "Conflict: No thermal data available for this area"
    ],
    "Parking Area": [
      "Parking Area seepage",
      "Conflict: No thermal data available for this area"
    ],
    "Common Bathroom": [
      "Common Bathroom Ceiling Dampness",
    

In [6]:
# Create formatted markdown report
report = f"""# Detailed Diagnostic Report (DDR)
*AI-Generated Property Inspection Analysis*

---

## Property Issue Summary
{ddr_json.get('property_issue_summary', 'Not Available')}

---

## Area-wise Observations
{ddr_json.get('area_wise_observations', 'Not Available')}

---

## Probable Root Cause
{ddr_json.get('probable_root_cause', 'Not Available')}

---

## Severity Assessment
{ddr_json.get('severity_assessment', 'Not Available')}

---

## Recommended Actions
{ddr_json.get('recommended_actions', 'Not Available')}

---

## Additional Notes
{ddr_json.get('additional_notes', 'Not Available')}

---

## Missing or Unclear Information
{ddr_json.get('missing_info', 'None identified')}

---

*Report generated using PyMuPDF + Groq LLM pipeline*
"""

print(report)

with open('DDR_Report.md', 'w', encoding='utf-8') as f:
    f.write(report)

# Download file
from google.colab import files
files.download('DDR_Report.md')

print("\n🎉 SUCCESS! DDR_Report.md downloaded to your computer.")

# Detailed Diagnostic Report (DDR)
*AI-Generated Property Inspection Analysis*

---

## Property Issue Summary
The property has several issues including dampness, leakage, and thermal anomalies. The main problems are related to the skirting, walls, and floors. Immediate action is required to prevent further damage.

---

## Area-wise Observations
{'Hall': ['Skirting level Dampness', 'Conflict: No thermal data available for this area'], 'Bedroom': ['Skirting level Dampness', 'Conflict: No thermal data available for this area'], 'Kitchen': ['Skirting level Dampness', 'Conflict: No thermal data available for this area'], 'Master Bedroom': ['Skirting level Dampness', 'Master bedroom wall damness', 'Conflict: No thermal data available for this area'], 'Parking Area': ['Parking Area seepage', 'Conflict: No thermal data available for this area'], 'Common Bathroom': ['Common Bathroom Ceiling Dampness', 'Condition of leakage at adjacent walls', 'Condition of leakage below WC', 'Leakage during: 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


🎉 SUCCESS! DDR_Report.md downloaded to your computer.
